In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp -r /content/gdrive/My\ Drive/WIDER/ /content

In [0]:
!mkdir data
%cd data

/content/data


In [0]:
%%capture
!unzip /content/gdrive/My\ Drive/WIDER/WIDER_train.zip
!unzip /content/gdrive/My\ Drive//WIDER/WIDER_test.zip
!unzip /content/gdrive/My\ Drive//WIDER/WIDER_val.zip
%cd ..

In [0]:
from random import shuffle
from PIL import Image
import numpy as np
import cv2
import zipfile
import io
import pickle
import os.path
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
def get_colors_for_classes(num_classes):
    """Return list of random colors for number of classes given."""
    # Use previously generated colors if num_classes is the same.
    if (hasattr(get_colors_for_classes, "colors") and
            len(get_colors_for_classes.colors) == num_classes):
        return get_colors_for_classes.colors

    hsv_tuples = [(x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(
        map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
            colors))
    random.seed(10101)  # Fixed seed for consistent colors across runs.
    random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    random.seed(None)  # Reset seed to default.
    get_colors_for_classes.colors = colors  # Save colors for future calls.
    return colors


def draw_boxes_(image, boxes, box_classes, class_names, scores=None):
   
    image = Image.fromarray(np.floor(image * 255 + 0.5).astype('uint8'))

    font = ImageFont.truetype(
        font='/content/YAD2K/font/FiraMono-Medium.otf',
        size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
    thickness = (image.size[0] + image.size[1]) // 300

    colors = get_colors_for_classes(len(class_names))

    for i, c in list(enumerate(box_classes)):
        box_class = class_names[c]
        box = boxes[i]
        if isinstance(scores, np.ndarray):
            score = scores[i]
            label = '{} {:.2f}'.format(box_class, score)
        else:
            label = '{}'.format(box_class)

        draw = ImageDraw.Draw(image)
        label_size = draw.textsize(label, font)

        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
        print(label, (left, top), (right, bottom))

        if top - label_size[1] >= 0:
            text_origin = np.array([left, top - label_size[1]])
        else:
            text_origin = np.array([left, top + 1])

        # My kingdom for a good redistributable image drawing library.
        for i in range(thickness):
            draw.rectangle(
                [left + i, top + i, right - i, bottom - i], outline=colors[c])
        draw.rectangle(
            [tuple(text_origin), tuple(text_origin + label_size)],
            fill=colors[c])
        draw.text(text_origin, label, fill=(0, 0, 0), font=font)
        del draw

    return np.array(image)

In [0]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    if os.path.isfile(anchors_path):
        with open(anchors_path) as f:
            anchors = f.readline()
            anchors = [float(x) for x in anchors.split(',')]
            return np.array(anchors).reshape(-1, 2)
    else:
        Warning("Could not open anchors file, using default.")
        return YOLO_ANCHORS

In [0]:
!git clone https://github.com/allanzelener/YAD2K.git

Cloning into 'YAD2K'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 2.35 MiB | 13.99 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [0]:
!wget https://gitlab.aiacademy.tw/aai/examples/raw/a71e50c55115627bd5d8ed2ddc5f22d26e131836/yolo_data/model_data/yolo_topless.h5

--2019-04-30 08:16:20--  https://gitlab.aiacademy.tw/aai/examples/raw/a71e50c55115627bd5d8ed2ddc5f22d26e131836/yolo_data/model_data/yolo_topless.h5
Resolving gitlab.aiacademy.tw (gitlab.aiacademy.tw)... 35.236.148.211
Connecting to gitlab.aiacademy.tw (gitlab.aiacademy.tw)|35.236.148.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202360872 (193M) [application/octet-stream]
Saving to: ‘yolo_topless.h5’

yolo_topless.h5     100%[===================>] 192.99M  11.2MB/s    in 19s     

2019-04-30 08:16:42 (10.3 MB/s) - ‘yolo_topless.h5’ saved [202360872/202360872]



In [0]:
!wget https://gitlab.aiacademy.tw/aai/examples/raw/a71e50c55115627bd5d8ed2ddc5f22d26e131836/yolo_data/model_data/yolo.h5

--2019-04-30 08:16:45--  https://gitlab.aiacademy.tw/aai/examples/raw/a71e50c55115627bd5d8ed2ddc5f22d26e131836/yolo_data/model_data/yolo.h5
Resolving gitlab.aiacademy.tw (gitlab.aiacademy.tw)... 35.236.148.211
Connecting to gitlab.aiacademy.tw (gitlab.aiacademy.tw)|35.236.148.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 204146732 (195M) [application/octet-stream]
Saving to: ‘yolo.h5’

yolo.h5             100%[===================>] 194.69M  13.8MB/s    in 16s     

2019-04-30 08:17:06 (12.4 MB/s) - ‘yolo.h5’ saved [204146732/204146732]



In [0]:
import os
import colorsys
import random
from PIL import Image, ImageDraw, ImageFont
from keras import optimizers as op
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from YAD2K.yad2k.models.keras_yolo import (preprocess_true_boxes, yolo_body,
                                     yolo_eval, yolo_head, yolo_loss)
from YAD2K.yad2k.utils.draw_boxes import draw_boxes


# Default anchor boxes
YOLO_ANCHORS = np.array(
    ((0.57273, 0.677385), (1.87446, 2.06253), (3.33843, 5.47434),
     (7.88282, 3.52778), (9.77052, 9.16828)))

Using TensorFlow backend.


In [0]:
%cd /content/gdrive/My\ Drive/WIDER/

/content/gdrive/My Drive/WIDER


In [0]:
train_box = np.load('final_boxes_preprocess.npz',allow_pickle=True)
train_det = np.load('detectandmatch.npz',allow_pickle=True)

In [0]:
val_box = np.load('val_final_boxes_preprocess.npz',allow_pickle = True)
val_det = np.load('val_detectandmatch.npz',allow_pickle = True)

In [0]:
val_box["boxes"].shape

(3225, 709, 5)

In [0]:
def batch_box(boxes, batch_size = 5,mode = "train"):
  i =0
  while True:
    bb = []
    while len(bb) < batch_size :
      a = boxes[i]
      i = i+1
      #print(len(bb))
      if(i == boxes.shape[0]):
        i =0;
        if mode == "eval":
          break;
      bb.append(a)

    yield(np.array(bb))

In [0]:
box_batch_train_27 = batch_box(train_box["boxes"], 27)
box_batch_train_9 = batch_box(train_box["boxes"], 9)
box_batch_train_3 = batch_box(train_box["boxes"], 3)
box_batch_train_1 = batch_box(train_box["boxes"], 1)

det_batch_train_27 = batch_box(train_det["detectors_mask"],27)
det_batch_train_9 = batch_box(train_det["detectors_mask"],9)
det_batch_train_3 = batch_box(train_det["detectors_mask"],3)
det_batch_train_1 = batch_box(train_det["detectors_mask"],1)

match_batch_train_27 = batch_box(train_det["matching_true_boxes"],27)
match_batch_train_9 = batch_box(train_det["matching_true_boxes"],9)
match_batch_train_3 = batch_box(train_det["matching_true_boxes"],3)
match_batch_train_1 = batch_box(train_det["matching_true_boxes"],1)

In [0]:
box_batch_val_25 = batch_box(val_box["boxes"], 25)
box_batch_val_5 = batch_box(val_box["boxes"], 5)
box_batch_val_3 = batch_box(val_box["boxes"], 3)
box_batch_val_1 = batch_box(val_box["boxes"], 1)

det_batch_val_25 = batch_box(val_det["detectors_mask"],25)
det_batch_val_5 = batch_box(val_det["detectors_mask"],5)
det_batch_val_3 = batch_box(val_det["detectors_mask"],3)
det_batch_val_1 = batch_box(val_det["detectors_mask"],1)

match_batch_val_25= batch_box(val_det["matching_true_boxes"],25)
match_batch_val_5 = batch_box(val_det["matching_true_boxes"],5)
match_batch_val_3 = batch_box(val_det["matching_true_boxes"],3)
match_batch_val_1 = batch_box(val_det["matching_true_boxes"],1)

In [0]:
import pandas as pd
tt =pd.read_csv("/content/gdrive/My Drive/WIDER/training_all_WIDER.csv")
#tt.head()
tt.columns = ["filename","boxes","Images"]
#tt.head()

In [0]:
vv =pd.read_csv("/content/gdrive/My Drive/WIDER/val_file_dataframe.csv")
#vv.head()

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_gen = ImageDataGenerator(rescale = 1./255.,)
val_gen = ImageDataGenerator(rescale = 1./255.,)
test_gen = ImageDataGenerator(rescale = 1./255.,)

In [0]:
test_generator = test_gen.flow_from_directory(directory="/content/data/WIDER_test/images",shuffle=True, batch_size=10,target_size=(416,416),class_mode =None, classes=None)


Found 16097 images belonging to 61 classes.


In [0]:
train_generator_27 = train_gen.flow_from_dataframe(dataframe=tt,directory="/content/data/WIDER_train/images",shuffle=False, xcol =0,batch_size=27,target_size=(416,416),class_mode =None, )
val_generator_25 = val_gen.flow_from_dataframe(dataframe=vv,directory="/content/data/WIDER_val/images",shuffle=False, xcol =0,batch_size=25,target_size=(416,416),class_mode =None, )


Found 12879 images.
Found 3225 images.


In [0]:
train_generator_3 = train_gen.flow_from_dataframe(dataframe=tt,directory="/content/data/WIDER_train/images",shuffle=False, xcol =0,batch_size=3,target_size=(416,416),class_mode =None, )
val_generator_3 = val_gen.flow_from_dataframe(dataframe=vv,directory="/content/data/WIDER_val/images",shuffle=False, xcol =0,batch_size=3,target_size=(416,416),class_mode =None, )


Found 12879 images.
Found 3225 images.


In [0]:
train_generator_9 = train_gen.flow_from_dataframe(dataframe=tt,directory="/content/data/WIDER_train/images",shuffle=False, xcol =0,batch_size=9,target_size=(416,416),class_mode =None, )
val_generator_5 = val_gen.flow_from_dataframe(dataframe=vv,directory="/content/data/WIDER_val/images",shuffle=False, xcol =0,batch_size=5,target_size=(416,416),class_mode =None, )
%cd /content

Found 12879 images.
Found 3225 images.
/content


In [0]:
def train_tot_gen(genX1,genX2,genX3,genX4):
  while True:
    X1 = genX1.__next__()
    X2 = genX2.__next__()
    X3 = genX3.__next__()
    X4 = genX4.__next__()
    y = np.zeros(len(X1))
    yield [X1, X2,X3,X4], y
    
    
def val_tot_gen(genX1,genX2,genX3,genX4):
  while True:
    X1 = genX1.__next__()
    X2 = genX2.__next__()
    X3 = genX3.__next__()
    X4 = genX4.__next__()
    y = np.zeros(len(X1))

    
    yield [X1, X2,X3,X4],y

In [0]:
def create_model(anchors, class_names, load_pretrained=True, freeze_body=True):
    '''
    returns the body of the model and the model
    # Params:
    load_pretrained: whether or not to load the pretrained model or initialize all weights
    freeze_body: whether or not to freeze all weights except for the last layer's
    # Returns:
    model_body: YOLOv2 with new output layer
    model: YOLOv2 with custom loss Lambda layer
    '''

    detectors_mask_shape = (13, 13, 5, 1)
    matching_boxes_shape = (13, 13, 5, 5)

    # Create model input layers.
    image_input = Input(shape=(416, 416, 3))
    boxes_input = Input(shape=(None, 5))
    detectors_mask_input = Input(shape=detectors_mask_shape)
    matching_boxes_input = Input(shape=matching_boxes_shape)

    # Create model body.
    yolo_model = yolo_body(image_input, len(anchors), len(class_names))
    topless_yolo = Model(yolo_model.input, yolo_model.layers[-2].output)

    if load_pretrained:
        # Save topless yolo:
        topless_yolo_path = os.path.join('/content', 'yolo_topless.h5')
        if not os.path.exists(topless_yolo_path):
            print("CREATING TOPLESS WEIGHTS FILE")
            yolo_path = os.path.join('/content', 'yolo.h5')
            model_body = load_model(yolo_path)
            model_body = Model(model_body.inputs, model_body.layers[-2].output)
            model_body.save_weights(topless_yolo_path)
        topless_yolo.load_weights(topless_yolo_path)

    if freeze_body:
        for layer in topless_yolo.layers:
            layer.trainable = False
    final_layer = Conv2D(len(anchors)*(5+len(class_names)), (1, 1), activation='linear')(topless_yolo.output)

    model_body = Model(image_input, final_layer)

    # Place model loss on CPU to reduce GPU memory usage.
    with tf.device('/cpu:0'):
        # TODO: Replace Lambda with custom Keras layer for loss.
        model_loss = Lambda(
            yolo_loss,
            output_shape=(1, ),
            name='yolo_loss',
            arguments={'anchors': anchors,
                       'num_classes': len(class_names)})([
                           model_body.output, boxes_input,
                           detectors_mask_input, matching_boxes_input
                       ])

    model = Model(
        [model_body.input, boxes_input, detectors_mask_input,
         matching_boxes_input], model_loss)

    return model_body, model

In [0]:
def train(model, class_names, anchors, trainimg,valimg, trainboxes,valboxes,
          train_detectors_mask, val_detectors_mask,
          train_matching_true_boxes,val_matching_true_boxes):
    
    model.compile(
        optimizer="adam", loss={
            'yolo_loss': lambda y_true, y_pred: y_pred
        },)  # This is a hack to use the custom loss function in the last layer.


    logging = TensorBoard()
    filepath="/gdrive/My\ Drive/WIDER/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,save_weights_only = True,)
    #checkpoint = ModelCheckpoint("trained_stage_3_best.h5", monitor='val_loss',
    #                             save_weights_only=True, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto')

    model.fit_generator(train_tot_gen(trainimg, trainboxes, train_detectors_mask, train_matching_true_boxes),
                        steps_per_epoch=(12879//27),
                        validation_data = val_tot_gen(valimg,valboxes,val_detectors_mask,val_matching_true_boxes),
                        validation_steps = 3225//25,epochs=5,callbacks=[logging])
    model.save_weights('trained_stage_1.h5')
    model.save_weights('/grive/My\ Drive/WIDER/trained_stage_1_1.h5')

  

def draw(model_body, class_names, anchors, image_data, image_set='val',
            weights_name='trained_stage_3_best.h5', out_path="output_images", save_all=True):
   
    if image_set == 'train':
        image_data = np.array([np.expand_dims(image, axis=0)
            for image in image_data[:int(len(image_data)*.9)]])
    elif image_set == 'val':
        image_data = np.array([np.expand_dims(image, axis=0)
            for image in image_data[int(len(image_data)*.9):]])
    elif image_set == 'all':
        image_data = np.array([np.expand_dims(image, axis=0)
            for image in image_data])
    else:
        ValueError("draw argument image_set must be 'train', 'val', or 'all'")
    # model.load_weights(weights_name)
    print(image_data.shape)
    model_body.load_weights(weights_name)

    # Create output variables for prediction.
    yolo_outputs = yolo_head(model_body.output, anchors, len(class_names))
    input_image_shape = K.placeholder(shape=(2, ))
    boxes, scores, classes = yolo_eval(
        yolo_outputs, input_image_shape, score_threshold=0.07, iou_threshold=0.0)

    # Run prediction on overfit image.
    sess = K.get_session()  # TODO: Remove dependence on Tensorflow session.

    if  not os.path.exists(out_path):
        os.makedirs(out_path)
    for i in range(len(image_data)):
        out_boxes, out_scores, out_classes = sess.run(
            [boxes, scores, classes],
            feed_dict={
                model_body.input: image_data[i],
                input_image_shape: [image_data.shape[2], image_data.shape[3]],
                K.learning_phase(): 0
            })
        print('Found {} boxes for image.'.format(len(out_boxes)))
        print("Boxes:",out_boxes)
        

        # Plot image with predicted boxes.
        image_with_boxes = draw_boxes_(image_data[i][0], out_boxes, out_classes,
                                    class_names, out_scores)
        print("--------------------------------------------")
        # Save the image:
        if save_all or (len(out_boxes) > 0):
             image = PIL.Image.fromarray(image_with_boxes)
             image.save(os.path.join(out_path,str(i)+'.png'))
            #cv2.imwrite(os.path.join(out_path,str(i)+'.png'),image_with_boxes)
        # To display (pauses the program):
        # plt.imshow(image_with_boxes, interpolation='nearest')
        # plt.show()


In [0]:
classes_path = "/content/gdrive/My Drive/WIDER/classes.txt"

class_names = get_classes(classes_path)
#anchors = get_anchors(anchors_path)

#data = np.load(data_path) # custom data saved as a numpy file.
#  has 2 arrays: an object array 'boxes' (variable length of boxes in each image)
#  and an array of images 'images'

#image_data, boxes = process_data(data['images'], data['boxes'])

anchors = YOLO_ANCHORS

#detectors_mask, matching_true_boxes = get_detector_mask(boxes, anchors)

model_body, model = create_model(anchors, class_names,freeze_body=False)

#model.summary()

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
train(
    model,
    class_names,
    anchors,
    train_generator_27,
    val_generator_25,
    box_batch_train_27,
    box_batch_val_25,
    det_batch_train_27,
    det_batch_val_25,
    match_batch_train_27,
    match_batch_val_25)

Epoch 1/5
477/477 [==============================] - 1497s 3s/step - loss: 393.9393 - val_loss: 3815.0439
Epoch 2/5
477/477 [==============================] - 1470s 3s/step - loss: 285.2336 - val_loss: 314.7425
Epoch 3/5
477/477 [==============================] - 1459s 3s/step - loss: 248.5030 - val_loss: 267.6052
Epoch 4/5
477/477 [==============================] - 1458s 3s/step - loss: 669.7827 - val_loss: 1093.9394
Epoch 5/5
477/477 [==============================] - 1460s 3s/step - loss: 641.7783 - val_loss: 492.8697


OSError: ignored

In [0]:
model.save_weights('/content/gdrive/My Drive/WIDER/trained_stage_1_1.h5')

In [0]:
def train2(model, class_names, anchors, trainimg,valimg, trainboxes,valboxes,
          train_detectors_mask, val_detectors_mask,
          train_matching_true_boxes,val_matching_true_boxes):
    '''
    retrain/fine-tune the model
    logs training with tensorboard
    saves training weights in current directory
    best weights according to val_loss is saved as trained_stage_3_best.h5
    '''
    """model.compile(
        optimizer=op.Adam(lr = 1e-3), loss={
            'yolo_loss': lambda y_true, y_pred: y_pred
        },)  # This is a hack to use the custom loss function in the last layer.


    """
    logging = TensorBoard(log_dir = '/content/gdrive/My Drive/WIDER/logs')
    filepath="/content/gdrive/My Drive/WIDER/epochs:{epoch:03d}-val_loss:{val_loss:.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True,save_weights_only = True,)
    #checkpoint = ModelCheckpoint("trained_stage_3_best.h5", monitor='val_loss',
    #                             save_weights_only=True, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto')

    """model.fit_generator(train_tot_gen(trainimg, trainboxes, train_detectors_mask, train_matching_true_boxes),
                        steps_per_epoch=(12879//27),
                        validation_data = val_tot_gen(valimg,valboxes,val_detectors_mask,val_matching_true_boxes),
                        validation_steps = 3225//25,epochs=5,callbacks=[logging])
    model.save_weights('trained_stage_1.h5')"""

    model_body, model = create_model(anchors, class_names, load_pretrained=False, freeze_body=False)

    model.load_weights('/content/gdrive/My Drive/WIDER/fin_epochs:042-val_loss:35.310.hdf5')

    model.compile(
        optimizer=op.Adam(lr = 1e-5), loss={
            'yolo_loss': lambda y_true, y_pred: y_pred
        })  # This is a hack to use the custom loss function in the last layer.
    model.fit_generator(train_tot_gen(trainimg, trainboxes, train_detectors_mask, train_matching_true_boxes),
                        steps_per_epoch=(12879//3),
                        validation_data = val_tot_gen(valimg,valboxes,val_detectors_mask,val_matching_true_boxes),
                        validation_steps = 3225//3,epochs=60,callbacks=[logging,checkpoint],initial_epoch =41)

    """model.fit_generator(tot_gen(image_data, boxes, detectors_mask, matching_true_boxes),
              np.zeros(len(image_data)),
              steps_per_epoch=12879/32,
              #batch_size=8,
              epochs=30,
              callbacks=[logging])"""

    model.save_weights('trained_stage_2.h5')

In [0]:
train2(
    model,
    class_names,
    anchors,
    train_generator_9,
    val_generator_5,
    box_batch_train_9,
    box_batch_val_5,
    det_batch_train_9,
    det_batch_val_5,
    match_batch_train_9,
    match_batch_val_5)

Epoch 42/60
1431/1431 [==============================] - 2213s 2s/step - loss: 60.3533 - val_loss: 51.1444

Epoch 00042: val_loss improved from inf to 51.14437, saving model to /content/gdrive/My Drive/WIDER/epochs:042-val_loss:51.144.hdf5
Epoch 43/60
1431/1431 [==============================] - 2241s 2s/step - loss: 57.7232 - val_loss: 51.9534

Epoch 00043: val_loss did not improve from 51.14437
Epoch 44/60
1431/1431 [==============================] - 2244s 2s/step - loss: 56.2754 - val_loss: 52.5175

Epoch 00044: val_loss did not improve from 51.14437
Epoch 45/60
1431/1431 [==============================] - 2244s 2s/step - loss: 55.1924 - val_loss: 52.7456

Epoch 00045: val_loss did not improve from 51.14437
Epoch 46/60
1431/1431 [==============================] - 2247s 2s/step - loss: 54.3117 - val_loss: 53.0587

Epoch 00046: val_loss did not improve from 51.14437
Epoch 47/60
1431/1431 [==============================] - 2244s 2s/step - loss: 53.5820 - val_loss: 53.7138

Epoch 00047:

KeyboardInterrupt: ignored

In [0]:
train2(
    model,
    class_names,
    anchors,
    train_generator_3,
    val_generator_3,
    box_batch_train_3,
    box_batch_val_3,
    det_batch_train_3,
    det_batch_val_3,
    match_batch_train_3,
    match_batch_val_3)

Epoch 42/60
4293/4293 [==============================] - 2904s 677ms/step - loss: 25.4733 - val_loss: 34.6233

Epoch 00042: val_loss improved from inf to 34.62326, saving model to /content/gdrive/My Drive/WIDER/epochs:042-val_loss:34.623.hdf5
Epoch 43/60
4293/4293 [==============================] - 2913s 679ms/step - loss: 25.1141 - val_loss: 34.5941

Epoch 00043: val_loss improved from 34.62326 to 34.59413, saving model to /content/gdrive/My Drive/WIDER/epochs:043-val_loss:34.594.hdf5
Epoch 44/60
4293/4293 [==============================] - 2891s 673ms/step - loss: 24.8764 - val_loss: 34.4761

Epoch 00044: val_loss improved from 34.59413 to 34.47607, saving model to /content/gdrive/My Drive/WIDER/epochs:044-val_loss:34.476.hdf5
Epoch 45/60
4293/4293 [==============================] - 2863s 667ms/step - loss: 24.6799 - val_loss: 34.4815

Epoch 00045: val_loss did not improve from 34.47607
Epoch 46/60
4293/4293 [==============================] - 2869s 668ms/step - loss: 24.5017 - val_lo

KeyboardInterrupt: ignored

In [0]:
x = test_generator.next()
draw(model_body,
class_names,
anchors,
x,
image_set='all', # assumes training/validation split is 0.9
weights_name='/content/gdrive/My Drive/WIDER/epochs:042-val_loss:35.310.hdf5',
save_all=False) 